In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision.models import resnet18

In [11]:
## modelの作成
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
print("model : ", model)

## 1層目の畳み込み層を1チャネルに変更
model.features[0][0] = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
    nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
    nn.ReLU6(inplace=True),
)

In [30]:
class MobileNetV2_audio(nn.Module):
  def __init__(self, dim=2048, pred_dim=512, arch=None):

    super(MobileNetV2_audio, self).__init__()

    # backboneの作成
    self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
    feature_dim = 1280

    # backboneの1層目の畳み込み層を変更
    self.backbone.features[0][0] = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
        nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU6(inplace=True),
    )

    # classifierをIdentity()に変更
    self.backbone.classifier = nn.Identity()

    # Projectorの設定
    self.projector = nn.Sequential(nn.Linear(feature_dim, feature_dim, bias=False),
                                    nn.BatchNorm1d(feature_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(feature_dim, feature_dim, bias=False),
                                    nn.BatchNorm1d(feature_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(feature_dim, dim, bias=True),
                                    nn.BatchNorm1d(dim, affine=False)
    )

    # Predictorの作成
    self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                  nn.BatchNorm1d(pred_dim),
                                  nn.ReLU(inplace=True),
                                  nn.Linear(pred_dim, dim))


  def forward(self, x):

    # backboneの出力
    z1 = self.backbone(x)
    print("z1.shape : ", z1.shape)

    # Projectorの出力
    z2 = self.projector(z1)
    print("z2.shape : ", z2.shape)

    # Predictorの出力
    z3 = self.predictor(z2)
    print("z3.shape : ", z3.shape)

    return z3



In [31]:
model = MobileNetV2_audio()
#print("model : ", model)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [33]:
a = torch.randn([10, 1, 224, 350])
y = model(a)
print(y.shape)

z1.shape :  torch.Size([10, 1280])
z2.shape :  torch.Size([10, 2048])
z3.shape :  torch.Size([10, 2048])
torch.Size([10, 2048])


In [2]:
class MobileNetV2_audio(nn.Module):
  def __init__(self, dim=2048, pred_dim=512, arch=None):

    super(MobileNetV2_audio, self).__init__()

    # backboneの作成
    self.backbone = torchvision.models.mobilenet_v2(pretrained=True)
    feature_dim = 1280

    # backboneの1層目の畳み込み層を変更
    self.backbone.features[0][0] = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
        nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU6(inplace=True),
    )

    # classifierをIdentity()に変更
    self.backbone.classifier = nn.Identity()

    # Projectorの設定
    self.projector = nn.Sequential(nn.Linear(feature_dim, feature_dim, bias=False),
                                    nn.BatchNorm1d(feature_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(feature_dim, feature_dim, bias=False),
                                    nn.BatchNorm1d(feature_dim),
                                    nn.ReLU(inplace=True),
                                    nn.Linear(feature_dim, dim, bias=True),
                                    nn.BatchNorm1d(dim, affine=False)
    )

    # Predictorの作成
    self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                  nn.BatchNorm1d(pred_dim),
                                  nn.ReLU(inplace=True),
                                  nn.Linear(pred_dim, dim))


  def forward(self, x1, x2=None):

    # backboneの出力
    self.outputs = []
    # for i, layer in enumerate(self.backbone.features):
    #   x = layer(x)
    #   print("x.shape : ", x.shape)
    #   #self.outputs.append(x)
    #   self.outputs.append(F.adaptive_avg_pool2d(x, 1))

    # for feat in self.outputs:
    #   print("feat.shape : ", feat.shape)
    # print(cfghj)

    feature = self.backbone(x1)
    print("feature.shape : ", feature.shape)

    # Projectorの出力
    z1 = self.projector(feature)
    print("z1.shape : ", z1.shape)

    # Predictorの出力
    p1 = self.predictor(z1)
    print("p1.shape : ", p1.shape)

    if x2 is None:
      return p1, z1.detach(), feature

    # encoderの出力
    feature2 = self.backbone(x2)

    # projectionの出力
    z2 = self.projector(feature2)

    # predictorの出力
    p2 = self.predictor(z2)

    return p1, p2, z1.detach(), z2.detach(),





In [3]:
model = MobileNetV2_audio()
print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 91.4MB/s]


MobileNetV2_audio(
  (backbone): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Sequential(
          (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runn

In [4]:
a = torch.randn([10, 1, 224, 350])
b = torch.randn([10, 1, 224, 350])

p1, p2, z1, z2 = model(a, b)


feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])


In [9]:
import torch.nn as nn
import torch.optim as optim

# モデルのインスタンスをCUDAに転送済み
#model = model.cuda()

# 交差エントロピー損失関数
# criterion = nn.CrossEntropyLoss()

# コサイン類似度
criterion = nn.CosineSimilarity()

# 最適化手法の設定（例: Adam）
optimizer = optim.Adam(model.parameters(), lr=0.001)

model = model
for i in range(5):

  mel1 = torch.randn([10, 1, 224, 350])
  mel1 = mel1
  #print("mel.shape : ", mel.shape)     # torch.Size([10, 1, 224, 350])

  mel2 = torch.randn([10, 1, 224, 350])
  mel2 = mel2

  label = torch.tensor([0, 1, 2, 3, 4, 5, 6, 5, 4, 3])
  label = label
  #print("label.shape : ", label.shape)   # torch.Size([10, 1])

  # 出力を獲得
  p1, p2, z1, z2 = model(mel1, mel2)
  # print("feature.shape : ", feature.shape)    # torch.Size([10, 1280])
  # print("z.shape : ", z.shape)                # torch.Size([10, 7])

  # 交差エントロピー損失計算
  loss_per_sample = -(criterion(p1, z2.detach()) + criterion(p2, z1.detach())) * 0.5
  loss = loss_per_sample.mean()
  print("loss : ", loss)


feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])
loss :  tensor(-0.0036, grad_fn=<MeanBackward0>)
feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])
loss :  tensor(0.0005, grad_fn=<MeanBackward0>)
feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])
loss :  tensor(0.0026, grad_fn=<MeanBackward0>)
feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])
loss :  tensor(0.0012, grad_fn=<MeanBackward0>)
feature.shape :  torch.Size([10, 1280])
z1.shape :  torch.Size([10, 2048])
p1.shape :  torch.Size([10, 2048])
loss :  tensor(0.0033, grad_fn=<MeanBackward0>)


In [25]:
backbone = torchvision.models.mobilenet_v2(pretrained=True)
print(backbone)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
## 教師あり用（出力一つ）を作成


class MobileNetV2_audio(nn.Module):
  def __init__(self, dim=2048, pred_dim=512, arch=None):

    super(MobileNetV2_audio, self).__init__()

    # backboneの作成
    self.backbone = torchvision.models.mobilenet_v2(pretrained=True)
    feature_dim = 1280

    # backboneの1層目の畳み込み層を変更
    self.backbone.features[0][0] = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
        nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
        nn.ReLU6(inplace=True),
    )

    # classifierをIdentity()に変更
    self.backbone.classifier = nn.Identity()

    self.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=False),
        nn.Linear(in_features=feature_dim, out_features=7, bias=True)
    )


  def forward(self, x1, x2=None):

    # backboneの出力
    self.outputs = []
    # for i, layer in enumerate(self.backbone.features):
    #   x = layer(x)
    #   print("x.shape : ", x.shape)
    #   #self.outputs.append(x)
    #   self.outputs.append(F.adaptive_avg_pool2d(x, 1))

    # for feat in self.outputs:
    #   print("feat.shape : ", feat.shape)
    # print(cfghj)

    feature = self.backbone(x1)
    #print("feature.shape : ", feature.shape)

    # Projectorの出力
    z1 = self.classifier(feature)
    #print("z1.shape : ", z1.shape)

    return feature, z1





In [40]:
model = MobileNetV2_audio()
print(model)

MobileNetV2_audio(
  (backbone): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Sequential(
          (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_runn

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [41]:
a = torch.randn([10, 1, 224, 350])

feature, z = model(a)

In [43]:
import torch.nn as nn
import torch.optim as optim

# モデルのインスタンスをCUDAに転送済み
model = model.cuda()

# 交差エントロピー損失関数
criterion = nn.CrossEntropyLoss()

# 最適化手法の設定（例: Adam）
optimizer = optim.Adam(model.parameters(), lr=0.001)

model = model.cuda()
for i in range(5):

  mel = torch.randn([10, 1, 224, 350])
  mel = a.cuda()
  #print("mel.shape : ", mel.shape)     # torch.Size([10, 1, 224, 350])

  label = torch.tensor([0, 1, 2, 3, 4, 5, 6, 5, 4, 3])
  label = label.cuda()
  #print("label.shape : ", label.shape)   # torch.Size([10, 1])

  # 出力を獲得
  feature, z = model(mel)
  # print("feature.shape : ", feature.shape)    # torch.Size([10, 1280])
  # print("z.shape : ", z.shape)                # torch.Size([10, 7])

  # 交差エントロピー損失計算
  loss = criterion(z, label)
  print("loss : ", loss)


loss :  tensor(2.0409, device='cuda:0', grad_fn=<NllLossBackward0>)
loss :  tensor(1.9958, device='cuda:0', grad_fn=<NllLossBackward0>)
loss :  tensor(2.0320, device='cuda:0', grad_fn=<NllLossBackward0>)
loss :  tensor(2.0408, device='cuda:0', grad_fn=<NllLossBackward0>)
loss :  tensor(1.9401, device='cuda:0', grad_fn=<NllLossBackward0>)
